In [4]:
%pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 12.8 MB/s eta 0:00:00m eta 0:00:010:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 16.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840629 sha256=19b7f43270fe5c9b2a39d7fc78c336298371eda796d09ff64039be7163523973
  Stored in directory: /Users/pratikjh/Library/Caches/pip/wheels/07/a0/a3/d24c94bf043ab5c7e38c30491199a2a11fef8d2584e6df7fb7
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [98]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, length, when
from pyspark.sql.types import IntegerType, BooleanType

In [100]:
spark = SparkSession.builder.appName("Chicago Nightlife Data Cleaning").getOrCreate()

In [102]:
# Load CSV into a Spark DataFrame
df = spark.read.csv("../data/raw/reddit_nightlife_data_20241202_002739.csv", header=True, inferSchema=True)

In [104]:
df.head()

Row(post_id='1glwebz', subreddit='Chicago', title='What’s your favorite Dive Bar in the city? ', text='Do you have a top 5? Top 3?', score=None, upvote_ratio=None, num_comments=None, created_utc=None, query_term=None, comment_id=None, parent_id=None, depth=None, full_text=None, mentioned_venues=None, text_length=None, has_venue_mention=None)

In [106]:
print(df.count())

10826


In [108]:
# Remove rows with missing or null values
df = df.dropna()

In [110]:
df.show()

+--------------------+--------------------+------------+--------------+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|             post_id|           subreddit|       title|          text|               score|        upvote_ratio|num_comments|         created_utc|          query_term|          comment_id|           parent_id|               depth|           full_text|    mentioned_venues|         text_length|   has_venue_mention|
+--------------------+--------------------+------------+--------------+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|**Food - 7.5.** A...| this was hit-or-...|   à la m

In [112]:
print(df.count())

10


In [54]:
# Drop duplicate rows based on post_id
df = df.dropDuplicates(["post_id"])

In [56]:
# Convert created_utc to a timestamp
df = df.withColumn("created_utc", col("created_utc").cast(IntegerType()))
df = df.withColumn("created_utc", (col("created_utc") / 1000).cast("timestamp"))

In [58]:
# Check cleaned data
df.show()

+--------------------+--------------------+------------+--------------+--------------------+--------------------+------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|             post_id|           subreddit|       title|          text|               score|        upvote_ratio|num_comments|created_utc|          query_term|          comment_id|           parent_id|               depth|           full_text|    mentioned_venues|         text_length|   has_venue_mention|
+--------------------+--------------------+------------+--------------+--------------------+--------------------+------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|**Food - 7.5.** A...| this was hit-or-...|   à la mode|         quail|        

In [82]:
print(df.count())

2
